In [1]:
import ee

# Authenticate to Earth Engine (you might need to run 'earthengine authenticate' command first)
ee.Authenticate()

Enter verification code: 4/1Adeu5BValcK-WOpf-7AgEpRHhepjSrjHww4paw0mrnApz1E6VJW19KW9qKo

Successfully saved authorization token.


In [8]:
ee.Initialize()

In [9]:
import ee
import geopandas as gpd
import pandas as pd
import datetime
import ast  # Import the ast module to parse the string coordinates
from tqdm import tqdm

In [16]:
# Load the Image Collection
dataset = ee.ImageCollection('JAXA/GPM_L3/GSMaP/v6/operational')

# Load the asset information from the CSV file
asset_info = pd.read_csv('asset_info.csv')

# Function to get weekly rainfall data
def get_weekly_rainfall(location_id, latitude, longitude, start_date, end_date):
    location_point = ee.Geometry.Point(longitude, latitude)
    aoi_geometry = location_point.buffer(10000)
    
    # Initialize an empty list to store data
    data = []
    
    # Split the date range into smaller intervals (e.g., 7 days)
    interval = datetime.timedelta(days=7)
    current_date = datetime.datetime.strptime(start_date, '%Y-%m-%d')
    
    while current_date <= datetime.datetime.strptime(end_date, '%Y-%m-%d'):
        # Define the current interval
        interval_start = current_date.strftime('%Y-%m-%d')
        current_date += interval
        interval_end = min(current_date, datetime.datetime.strptime(end_date, '%Y-%m-%d')).strftime('%Y-%m-%d')
        
        # Filter the dataset for the current interval and location
        filtered_precipitation = dataset.filterDate(interval_start, interval_end).filterBounds(aoi_geometry)
        
        # Function to calculate weekly precipitation
        def calculate_weekly_precip(image):
            weekly_precip = image.reduceRegion(
                reducer=ee.Reducer.sum(),
                geometry=aoi_geometry,
                scale=11500
            ).get('hourlyPrecipRate')
            return image.set('weeklyPrecipitation', weekly_precip)
        
        rainfall_data = filtered_precipitation.map(calculate_weekly_precip)
        
        # Extract data for the current interval
        for image in rainfall_data.getInfo()['features']:
            date_start = datetime.datetime.fromtimestamp(image['properties']['system:time_start'] / 1000)
            year_week = date_start.strftime('%Y-%U')
            weekly_precipitation = image['properties'].get('weeklyPrecipitation', 'N/A')
            data.append({'Year_Week': year_week, 'Location ID': location_id, 'Precipitation (mm/week)': weekly_precipitation, 'Latitude': latitude, 'Longitude': longitude})
    
    return data

# Create a directory to save individual CSV files
output_dir = '/content/individual_location_data'
os.makedirs(output_dir, exist_ok=True)

# Loop through each row of asset_info with tqdm for a progress bar
for index, row in tqdm(asset_info.iterrows(), total=len(asset_info)):
    location_id = row['Location']
    latitude = row['Latitude']
    longitude = row['Longitude']
    start_date = '2018-01-01'
    end_date = '2023-09-09'
    
    # Get weekly rainfall data
    rainfall_data = get_weekly_rainfall(location_id, latitude, longitude, start_date, end_date)
    
    # Create a DataFrame from the data
    location_data = pd.DataFrame(rainfall_data)
    
    # Save the data to an individual CSV file for each location
    output_filename = os.path.join(output_dir, f'{location_id}_rainfall_data.csv')
    location_data.to_csv(output_filename, index=False)

print('All data saved to individual CSV files in the directory:', output_dir)

 71%|█████████████████████████████████████████████████████████▊                       | 5/7 [1:15:56<30:22, 911.27s/it]


ConnectionError: ('Connection aborted.', ConnectionAbortedError(10053, 'An established connection was aborted by the software in your host machine', None, 10053, None))